In [11]:
import math
import numpy as np
import math
from datasets import load_dataset

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import json
import os
import time
import random
import ast

In [12]:
# Leggi il dataset originale una sola volta
silver_dataset = pd.read_csv('../datasets/silver_unicum.csv')
gold_dataset = pd.read_csv('../datasets/gold_unicum.csv')


In [13]:
def to_quantili(train_dataset, val_dataset, in_column_name, out_column_name, n_quantili):
    #Taken a numerical column as input, <n_quantili> bins are created. 
    #The "quantilization" is performed based just on train_dataset values, to avoid data leakage
    #The bins are then applied both to the train and val dataset
    #Finally the old numerical columns are dropped
    
    bins_edges = pd.qcut(train_dataset[in_column_name], q=n_quantili, retbins=True)[1]
    train_dataset[out_column_name] = pd.cut(train_dataset[in_column_name], bins=bins_edges, include_lowest=True, duplicates='drop')
    val_dataset[out_column_name] = pd.cut(val_dataset[in_column_name], bins=bins_edges, include_lowest=True, duplicates='drop')

    train_dataset.drop(columns=in_column_name, inplace=True)
    val_dataset.drop(columns=in_column_name, inplace=True)

    return train_dataset, val_dataset


In [14]:
#If entry == something i just punt one, regarding of the content
str_to_one = lambda x: 0 if len(x) == 1 else 1
silver_dataset['countryLabel'] = silver_dataset['countryLabel'].apply(str_to_one)
silver_dataset['subclass_ofLabel'] = silver_dataset['subclass_ofLabel'].apply(str_to_one)
gold_dataset['countryLabel'] = gold_dataset['countryLabel'].apply(str_to_one)
gold_dataset['subclass_ofLabel'] = gold_dataset['subclass_ofLabel'].apply(str_to_one)


In [15]:
label_columns = ['label_cultural agnostic', 'label_cultural exclusive', 'label_cultural representative']

In [16]:
print(silver_dataset.columns)
print(gold_dataset.columns)

Index(['item', 'name', 'type', 'category', 'subcategory', 'label',
       'countryLabel', 'subclass_ofLabel', 'text_distribution', 'std_text',
       'avg_text', 'len', 'entropy_text', 'gini_text', 'sum_over_texts',
       'ref_distribution', 'std_ref', 'avg_ref', 'sum_over_ref', 'entropy_ref',
       'gini_ref', 'h_adj_descr', 'h_nat_descr', 'h_adj_engtext',
       'h_nat_engtext'],
      dtype='object')
Index(['item', 'name', 'type', 'category', 'subcategory', 'label',
       'countryLabel', 'subclass_ofLabel', 'text_distribution', 'std_text',
       'avg_text', 'len', 'entropy_text', 'gini_text', 'sum_over_texts',
       'ref_distribution', 'std_ref', 'avg_ref', 'sum_over_ref', 'entropy_ref',
       'gini_ref', 'h_adj_descr', 'h_nat_descr', 'h_adj_engtext',
       'h_nat_engtext'],
      dtype='object')


In [17]:
#drop unnecessary columns
columns_to_quantili = ['len']

unnecessary_columns =  [ 'item', 'name', 'text_distribution', 'ref_distribution', 'std_text', 'avg_text', 'entropy_text', 'gini_text', 'sum_over_texts',
                         'sum_over_ref', 'entropy_ref', 'gini_ref', 'h_adj_engtext',
                        'h_nat_engtext'
                    ]
silver_dataset = silver_dataset.drop(columns=unnecessary_columns)
gold_dataset = gold_dataset.drop(columns=unnecessary_columns)

In [18]:
#Divide numerical features in classes based on quantili

for feature in columns_to_quantili:
    silver_dataset, gold_dataset = to_quantili(silver_dataset, gold_dataset, feature, "bin_"+feature, 4)

In [19]:
print(silver_dataset.columns)

Index(['type', 'category', 'subcategory', 'label', 'countryLabel',
       'subclass_ofLabel', 'std_ref', 'avg_ref', 'h_adj_descr', 'h_nat_descr',
       'bin_len'],
      dtype='object')


In [20]:
from sklearn.ensemble import ExtraTreesClassifier  # Cambiato da RandomForest

categorical_columns = silver_dataset.columns
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(silver_dataset[categorical_columns])
encoded_train = encoder.transform(silver_dataset[categorical_columns])
encoded_eval = encoder.transform(gold_dataset[categorical_columns])
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_columns), index=silver_dataset.index)
encoded_eval_df = pd.DataFrame(encoded_eval, columns=encoder.get_feature_names_out(categorical_columns), index=gold_dataset.index)

training_dataset = silver_dataset.drop(columns=categorical_columns)
training_dataset = pd.concat([training_dataset, encoded_train_df], axis=1)
evaluation_dataset = gold_dataset.drop(columns=categorical_columns)
evaluation_dataset = pd.concat([evaluation_dataset, encoded_eval_df], axis=1)

X_train = training_dataset.drop(columns=label_columns)
y_train = training_dataset[label_columns]
X_test = evaluation_dataset.drop(columns=label_columns)
y_test = evaluation_dataset[label_columns]

from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # puoi modificarli a piacere
    activation='relu',
    solver='adam',
    max_iter=300,+
    59640
    random_state=42
)

mlp_model.fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)

score = f1_score(y_test, y_pred, average='weighted')
acc = accuracy_score(y_test, y_pred)
print("ACCURACY (Neural Net):", acc)


ACCURACY (Neural Net): 0.6187290969899666
